In [7]:
using Pkg
Pkg.instantiate()
import TAMode
using Turing

In [68]:
tpoints = 10.0 .^ range(-6.0, stop = 2.0, length = 10)

@model test(expData, tps, ::Type{TV}=Vector{Float64}) where {TV} = begin    
    paramsA ~ MvLogNormal(fill(-6.0, 2), 0.01)
    paramsB ~ Truncated(LogNormal(-1.0, 0.01), 0.0, 1.0)
    paramsC ~ MvLogNormal(fill(-6.0, 12), 0.01)
    
    params = vcat(paramsA, paramsB, paramsC)

    #data = TAMode.getAutocrine(params)
    data = TAMode.runTAM(tps, params, 10.0)
    pYAXL = TAMode.pY .* TAMode.recpSpecific[1]
    tData = data * pYAXL
    resids = expData .- tData #scale?
    resids ~ MvNormal(zeros(length(resids)), ones(length(resids))*std(resids))
end

(::DynamicPPL.ModelGen{(:expData, :tps, :TV),var"###test#1835",NamedTuple{(:TV,),Tuple{Type{Array{Float64,1}}}}}) (generic function with 1 method)

In [ ]:
kTps = [0, 0.5, 1, 5, 10];
pyK = [4.1, 3.6, 5.3, 11.3, 11.6];
samp = sample(test(pyK, kTps), HMC(0.01, 5), 10)